In [1]:
import random
import utils
import numpy as np
import matplotlib.pyplot as plt

stack_size=10
random_agent_qtable=None
greedy_agent_qtable=None

# LeducGame class definition

In [2]:
class LeducGame:
    deck = []
    actions = [0,1,2] #0 is fold, 1 is check, 2 push
    firstplayer=None; #0 if player1 and 1 if player2
    hand_player1=0;
    hand_player2=0;
    boardcard=0;
    result=0;
    step_number=0
    roundGame=0 #0 preflop, 1 postflop
    game_round=0
    pot=None
    stack1=None
    stack2=None
    lastAction=None
    current_player=None
    game_is_over=None #0 game is not over, #1 game is over
    actions_hist=[]
    #inititate a game
    def __init__(self):
        self.deck = [0,0,1,1,2,2]
        
        #deal card to game from deck
        self.hand_player1=utils.choose_and_remove(self.deck)
        self.hand_player2=utils.choose_and_remove(self.deck)
        self.boardcard=utils.choose_and_remove(self.deck)
        self.result=self.get_result()
        self.firstplayer=random.randrange(0,2)
        #self.firstplayer=0
        self.step_number=0
        self.roundGame=0
        self.game_round=0
        self.pot=0
        self.stack1=stack_size
        self.stack2=stack_size
        self.current_player=self.firstplayer
        self.game_is_over=0 #0 not over, 1 over
        
        self.pot=2
        self.stack1=self.stack1-1
        self.stack2=self.stack2-1
        
    #allow to print leduc game state
    def __str__(self):
        return "FirstPlayer = {} \nHand1 = {} \nHand2 = {} \nBoard = {} \nDeck = {}\nResult = {}\nStack1={}\nStack2={}\nPot={}\nStep={}\nRound={}\nGameIsOver={}\nCurrent_player{}\n\n".format(self.firstplayer,self.hand_player1,self.hand_player2,self.boardcard,self.deck, self.result,self.stack1,self.stack2,self.pot,self.step_number,self.game_round,self.game_is_over, self.current_player)
     
    def get_firstplayer(self):
        return self.firstplayer
        
    def get_hand_player1(self):
        return self.hand_player1
    
    def get_hand_player2(self):
        return self.hand_player2
        
    def get_boardcard(self):
        return self.boardcard
        
    def get_current_player(self):
        return self.current_player
        
    def is_game_over(self):
        return self.game_is_over
    
    def get_game_round(self):
        return self.game_round
        
    #result() : 
    # 0  -> draw
    # 1  -> player1 win
    #-1  -> player2 win
    def get_result(self): #determine the best hand
        #Pairs
        if (self.hand_player1==self.boardcard):
            result=1
        elif (self.hand_player2==self.boardcard):
            result=-1
        #Highest card
        elif (self.hand_player1>self.hand_player2):
            result=1
        elif(self.hand_player1<self.hand_player2):
            result=-1
        #Draw
        else:
            result=0
        return result    
    
    def get_allowed_actions(self, action):
        if (action==0):
            return None
        elif (action==1):        
            return [0,1,2]
        elif (action==2):
            return [0, 2]
        
    def step_prime(self, action):
        old_round=self.game_round
        gain=0
        
        #QAGENT
        if(self.current_player==0):
            if(action==0):
                gain=-1
                self.game_is_over=1
            elif(action==2):
                self.stack1=0

            #step1
            if(self.step_number==0):
                if(action==1):
                    self.lastAction=1
    
                if(action==2):
                    self.pot=12
                    self.lastAction=2
                self.step_number=1
            #step2        
            elif(self.step_number==1):
                if(action==1):
                    if(self.lastAction==1):
                        if(self.game_round==1):
                            self.game_is_over=1
                            
                            if(self.get_result()==1):
                                gain=1
                            if(self.get_result()==-1):
                                gain=-1
                            
                        self.game_round=1
                        self.step_number=0
                if(action==2):
                    if(self.lastAction==2):
                        self.pot=20
                        self.game_is_over=1
                        
                        if(self.get_result()==1):
                            gain=10
                        if(self.get_result()==-1):
                            gain=-10
                
                            
                    if(self.lastAction==1):
                        self.pot=12
                        self.step_number=2

            #step3
            elif(self.step_number==2):
                if(action==2):
                    self.pot=20
                    self.game_is_over=1
                    
                    if(self.get_result()==1):
                        gain=10
                    if(self.get_result()==-1):
                        gain=-10
                        
        #OPPONENT          
        elif(self.current_player==1):
            if(action==0):
                gain=1
                self.game_is_over=1
            elif(action==2):
                self.stack2=0

            #step1
            if(self.step_number==0):
                if(action==1):
                    self.lastAction=1
                if(action==2):
                    self.pot=12
                    self.lastAction=2
                self.step_number=1
                    
            #step2        
            elif(self.step_number==1):
                if(action==1):
                    if(self.lastAction==1):
                        if(self.game_round==1):
                            self.game_is_over=1
                            
                            if(self.get_result()==1):
                                gain=1
                            if(self.get_result()==-1):
                                gain=-1
                            
                        self.game_round=1
                        self.step_number=0
                if(action==2):
                    if(self.lastAction==2):
                        self.pot=20
                        self.game_is_over=1
                        
                        if(self.get_result()==1):
                            gain=10
                        if(self.get_result()==-1):
                            gain=-10
                            
                    if(self.lastAction==1):
                        self.pot=12
                        self.step_number=2
            #step3
            elif(self.step_number==2):
                if(action==2):
                    self.pot=20
                    self.game_is_over=1
                    
                    if(self.get_result()==1):
                        gain=10
                    if(self.get_result()==-1):
                        gain=-10
            
        allowed_actions= self.get_allowed_actions(action)
        #print("allowed_actions in step_prime: ", allowed_actions)
        #print("action= ", action)
        #print(self)
        if(old_round==self.game_round):
            if(self.current_player==0):
                self.current_player=1
            elif(self.current_player==1):
                self.current_player=0
            #self.current_player=not(self.current_player)
        else:
            self.current_player=self.firstplayer   
        
        return gain,self.current_player, allowed_actions          

# Environment

In [3]:
class Environment:
    agent=None
    game=None
    opponent_action=None
    actions_hist=[]
    
    def __init__(self,agent):
        self.game=LeducGame()
        self.agent=agent
        
    def __str__(self):
        return "{} \nOpponent_Action = {}\nStack = {}".format(self.game,self.agent.get_action(),self.stack)
    
    def get_actions_hist(self):
        return self.actions_hist
    
    def set_opponent_action(self, action):
        self.opponent_action=action
    
    def get_state(self):
        hand1=self.game.get_hand_player1()
        first=self.game.get_firstplayer()
        boardcard= self.game.get_boardcard()
        
        if self.game.game_round == 0:
            if first is 0:
                return hand1
            elif(first==1 and self.opponent_action==1):
                return hand1+3
            elif(first==1 and self.opponent_action==2):
                return hand1+6
            
        elif self.game.game_round == 1:
            if first is 0:
                if hand1==0:
                    return boardcard+9
                elif hand1== 1:
                    return boardcard+12
                elif hand1== 2:
                    return boardcard+15
            else:
                if (self.opponent_action==1):
                    if hand1==0:
                        return boardcard+18
                    elif hand1==1:
                        return boardcard+21
                    elif hand1==2:
                        return boardcard+24
                if(self.opponent_action==2):
                    if hand1==0:
                        return boardcard+27
                    elif hand1==1:
                        return boardcard+30
                    elif hand1==2:
                        return boardcard+33
    
    def reset(self):
        self.game=LeducGame()
        self.actions_hist=[]
    
    def step(self, qagent_action):
        r=0
        state=None
        last_game_round=self.game.get_game_round()
        self.actions_hist.append([self.get_state(),qagent_action])
        r,current_player,allowed_actions=self.game.step_prime(qagent_action)
        if(allowed_actions==None):
            self.game.game_is_over=1
        if(self.game.is_game_over()==0):
            if(last_game_round==self.game.get_game_round()):
                self.agent.set_action(allowed_actions,qagent_action, self.game.get_game_round(), self.game.get_hand_player2(),self.game.get_boardcard())
            else:
                self.agent.set_action(allowed_actions,None, self.game.get_game_round(), self.game.get_hand_player2(),self.game.get_boardcard())
            self.opponent_action=self.agent.get_action()
            #print( "agent action", self.opponent_action)
            r,current_player,allowed_actions= self.game.step_prime(self.opponent_action)
            if(self.game.is_game_over()==1):
                state=None
            else:
                state=self.get_state()
        
        return r, allowed_actions, state
            

## Random Agent

In [4]:
class RandAgent():
    
    action=None
          
        
    def __init__(self):
        #qtable creation
        self.set_action([0,1,2], 0,0,0,0)
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        self.action=random.choice(allowed_actions)
        
    def get_action(self):
        return self.action

## Greedy Agent

In [5]:
class GreedyAgent():
    
    action=None
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        if game_round == 0:
            if qagent_action == None:
                if hand == 0:
                    action_list=[2,0,0,1,1,1,1,1,1,1] #10% of the time it will push, 20% of the time it will fall and 70% of the time it will check 
                    self.action=random.sample(action_list,1)[0]
                elif hand == 1:
                    action_list=[2,2,2,1,1,1,1,1,1,1] 
                    self.action=random.sample(action_list,1)[0]
                elif hand == 2:
                    action_list=[2,2,2,2,1,1,1,1,1,1] 
                    self.action=random.sample(action_list,1)[0]
            elif qagent_action == 1:
                if hand == 0:
                    action_list=[0,0,0,1,1,1,1,1,1,1] 
                    self.action=random.sample(action_list,1)[0]
                elif hand == 1:
                    self.action=1
                elif hand == 2:
                    action_list=[2,2,1,1,1,1,1,1,1,1] 
                    self.action=random.sample(action_list,1)[0]
            elif qagent_action == 2:
                if hand == 0:
                    self.action= 0
                elif hand == 1:
                    action_list=[2,2,0,0,0,0,0,0,0,0] 
                    self.action=random.sample(action_list,1)[0]
                elif hand == 2:
                    self.action= 2
        elif game_round == 1:
            if qagent_action == None:
                if hand == 0:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        action_list=[0,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        action_list=[0,1] 
                        self.action=random.sample(action_list,1)[0]
                elif hand == 1:
                    if boardcard == 0:
                        action_list=[2,2,1,1,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[2,2,2,2,2,2,2,2,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        action_list=[2,2,2,1,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                elif hand == 2:
                    if boardcard == 0:
                        action_list=[0,2,2,1,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[0,0,2,1,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        self.action= 2
            elif qagent_action == 1:
                if hand == 0:
                    if boardcard == 0:
                        action_list=[2,2,2,2,2,2,2,2,2,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[0,0,1,1,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        action_list=[0,0,0,0,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                elif hand == 1:
                    if boardcard == 0:
                        action_list=[0,0,1,1,1,1,1,1,1,2] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[2,2,2,2,2,2,2,2,2,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        action_list=[0,0,0,0,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                elif hand == 2:
                    if boardcard == 0:
                        action_list=[0,2,2,2,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[0,0,2,2,1,1,1,1,1,1] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        action_list=[2,2,2,2,2,2,2,2,2,1] 
                        self.action=random.sample(action_list,1)[0]
            elif qagent_action == 2:
                if hand == 0:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        self.action= 0
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 1:
                    if boardcard == 0:
                        self.action= 0
                    elif boardcard == 1:
                        self.action= 2
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 2:
                    if boardcard == 0:
                        action_list=[2,2,2,0,0,0,0,0,0,0] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 1:
                        action_list=[2,2,0,0,0,0,0,0,0,0] 
                        self.action=random.sample(action_list,1)[0]
                    elif boardcard == 2:
                        self.action= 2
        
    def get_action(self):
        return self.action

## Human Agent

In [6]:
class HumanAgent():
    
    action=None
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        if (game_round==0):
            if (qagent_action==None):
                print("Your card = ",hand,"\n\n")
            else:
                print("Your card =", hand ,"\nAction of the QAgent = ",qagent_action,"\n\n")
        elif (game_round==1):
            if (qagent_action==None):
                print("Your card =", hand ,"\nBoardcard = ",boardcard ,"\n\n")
            else:
                print("Your card =", hand, "\nAction of the QAgent = ",qagent_action,"\nBoardcard = ", boardcard," \n\n")
        self.action = int(input("Please enter an action :(0,1 or 2)\n"))
        
        print("\n\n")
        
        
    def get_action(self):
        return self.action

## QAgent

In [7]:
class QAgent():
    qtable=[]
 # Agent class definition   state=None
    learning_rate=0.1
    
    state=None #0-5
    gamma=1
    state_number=36
    state=None 
    perf=[]
    
    def __init__(self):
        #qtable creation
        self.qtable=np.zeros((self.state_number,3))
        self.perf=[]
        
    #allow to print leduc game state
    def __str__(self):
        return "State = {} \nQTable = {} \nLearning rate = {}".format(self.state,self.qtable,self.learning_rate)
    
    def explore_action(self, allowed_actions):
        action=random.choice(allowed_actions)
        return action
    
    def exploit_action(self, allowed_actions):
        print("allowed_actions= ", allowed_actions)
        action=utils.get_max_list(self.qtable[self.state], allowed_actions)
        return action
      
    def set_state(self, state):
        self.state=state    
        
    def set_qtable(self,qtable):
        self.qtable=qtable
        
    def set_perf(self,perf):
        self.perf.append(perf)

    def update(self,reward, actions_hist):
        #print("UPDATE action= {} \n reward = {} \n next_state = {}\n".format(action,reward,next_state))
        new_value = 0
        
        #print ("Update: \n")
        #print ("reward = ", reward)
        #print ("actions_hist = ", actions_hist)
        if(len(actions_hist)==1):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = new_value
            #print(actions_hist)
        if(len(actions_hist)==2):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[1][0], actions_hist[1][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[1][0], actions_hist[1][1]] = new_value

            back = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * (0 + (self.gamma/2) * reward)
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = back

        if(len(actions_hist)==3):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[2][0], actions_hist[2][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[2][0], actions_hist[2][1]] = new_value

            back = (1 - self.learning_rate) * self.qtable[actions_hist[1][0], actions_hist[1][1]] +  self.learning_rate * (0 + (self.gamma/2) * reward)
            self.qtable[actions_hist[1][0], actions_hist[1][1]] = back

            back_prime = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * (0 + (self.gamma/4) * reward)
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = back_prime

# Training and Testing parameters

In [8]:
epochs_number=100000
evaluate_every=30
test_number=1000

## qAgent vs RandAgent

In [9]:
randAgent=RandAgent()
env=Environment(randAgent)
qagent=QAgent()

for i in range(epochs_number):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,0,0,0,0)
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
        if(env.game.is_game_over()==0):
            qagent_action=qagent.explore_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)
        else:
            env.reset()
            allowed_actions=[0,1,2]
            first_time=True
    
    env.reset()
    
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)
random_agent_qtable=qagent.qtable

[[-0.201  0.355 -0.185]
 [-0.669  0.5    0.559]
 [-1.027  0.653  3.332]
 [-1.    -0.058 -1.063]
 [-1.     0.728 -0.853]
 [-1.     0.716  2.425]
 [-0.609  0.    -2.078]
 [-1.     0.    -2.241]
 [-0.739  0.     6.537]
 [-1.     1.121  5.85 ]
 [-1.    -0.419 -1.605]
 [-1.    -0.412 -3.231]
 [-1.    -0.389 -1.232]
 [-1.     0.958  5.288]
 [-1.     0.665  1.416]
 [-1.     0.342  1.459]
 [-1.     0.791  2.51 ]
 [-1.     1.304  4.322]
 [-0.654  0.996  5.753]
 [-0.266 -0.868 -3.031]
 [-0.655 -0.621 -1.176]
 [-0.453 -0.835 -3.081]
 [-0.162  1.229  5.52 ]
 [-0.629  0.446  0.126]
 [-0.081  0.295  0.777]
 [-0.492  0.029  3.261]
 [-0.501  1.439  4.738]
 [-0.92   0.     9.997]
 [-0.976  0.    -8.955]
 [-0.784  0.    -7.139]
 [-0.738  0.    -5.138]
 [-0.754  0.     9.989]
 [-0.999  0.     4.47 ]
 [-0.944  0.     2.094]
 [-0.937  0.     2.236]
 [-0.861  0.     9.98 ]]


## qAgent vs GreedyAgent

In [10]:
greedyAgent=GreedyAgent()
env=Environment(greedyAgent)
qagent=QAgent()

for i in range(epochs_number):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,None, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())

            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
            
        if(env.game.is_game_over()==0):
            qagent_action=qagent.explore_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)
        else:
            env.reset()
            allowed_actions=[0,1,2]
            first_time=True
    
    env.reset()
    
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)
greedy_agent_qtable=qagent.qtable

[[ -0.718   0.127  -1.478]
 [ -0.994  -0.354  -0.441]
 [ -1.127  -0.479   1.276]
 [ -1.     -0.539  -0.35 ]
 [ -1.     -0.045   1.485]
 [ -1.      0.214   1.254]
 [ -1.      0.     -4.838]
 [ -0.904   0.     -1.658]
 [ -1.      0.      1.463]
 [ -1.      0.947   1.616]
 [ -1.     -1.173  -3.88 ]
 [ -1.     -0.931  -2.593]
 [ -1.     -0.56   -2.674]
 [ -1.      1.198   2.018]
 [ -1.     -0.003  -2.017]
 [ -1.     -0.65   -3.108]
 [ -1.     -0.875  -3.443]
 [ -1.      1.      1.   ]
 [ -0.997   0.979   1.143]
 [ -0.981  -0.904  -0.374]
 [ -0.797  -0.638   0.693]
 [ -0.998  -0.5    -2.744]
 [ -0.765   0.998   1.487]
 [ -0.744   0.444   0.915]
 [ -0.904   0.335  -0.255]
 [ -0.77    0.377   0.189]
 [ -0.978   0.88    0.947]
 [ -0.987   0.      9.995]
 [ -0.98    0.    -10.   ]
 [ -1.      0.    -10.   ]
 [ -1.      0.     -9.236]
 [ -0.613   0.      9.282]
 [ -1.      0.     -8.678]
 [ -1.      0.     -3.514]
 [ -1.      0.     -9.489]
 [ -0.999   0.      9.903]]


## qAgent (trained with a random agent) vs HumanAgent

In [11]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()
qagent.set_qtable(random_agent_qtable)

print("How to play?\n\nFor the actions:\n\t- 0 : Fold\n\t- 1 : Check\n\t- 2 : Push\n\nFor the cards:\n\t- 0 : Jack\n\t- 1 : Queen\n\t- 2 : King\n\n ")
for i in range(1):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,None, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
            if(env.game.is_game_over()==1):
                print("You loose!! He has a:", env.game.get_hand_player1(), "\n\n")
        if(env.game.is_game_over()==0):
            qagent_action=qagent.exploit_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                if(reward<0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nCongratulation, you win !!!\n\n" )
                elif(reward>0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nSorry, you loose !!!\n\n" )
                elif(reward==0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nIt's a draw !!!\n\n" )
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)

    
    env.reset()
    

How to play?

For the actions:
	- 0 : Fold
	- 1 : Check
	- 2 : Push

For the cards:
	- 0 : Jack
	- 1 : Queen
	- 2 : King

 
allowed_actions=  [0, 1, 2]
Your card = 0 
Action of the QAgent =  2 


Please enter an action :(0,1 or 2)
0



His action was:  2 
He has a:  1 
Sorry, you loose !!!




## qAgent (trained with a greedy agent) vs HumanAgent

In [13]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()
qagent.set_qtable(greedy_agent_qtable)

print("How to play?\n\nFor the actions:\n\t- 0 : Fold\n\t- 1 : Check\n\t- 2 : Push\n\nFor the cards:\n\t- 0 : Jack\n\t- 1 : Queen\n\t- 2 : King\n\n ")
for i in range(1):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,None, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
            if(env.game.is_game_over()==1):
                print("You loose!! He has a:", env.game.get_hand_player1(), "\n\n")
        if(env.game.is_game_over()==0):
            qagent_action=qagent.exploit_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                if(reward<0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nCongratulation, you win !!!\n\n" )
                elif(reward>0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nSorry, you loose !!!\n\n" )
                elif(reward==0):
                    print("His action was: ", qagent_action, "\nHe has a: ",env.game.get_hand_player1(), "\nIt's a draw !!!\n\n" )
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)

    
    env.reset()
    

How to play?

For the actions:
	- 0 : Fold
	- 1 : Check
	- 2 : Push

For the cards:
	- 0 : Jack
	- 1 : Queen
	- 2 : King

 
allowed_actions=  [0, 1, 2]
Your card = 2 
Action of the QAgent =  1 


Please enter an action :(0,1 or 2)
2



allowed_actions=  [0, 2]


AttributeError: 'numpy.ndarray' object has no attribute 'remove'